In [1]:
pip install imblearn

Note: you may need to restart the kernel to use updated packages.


In [31]:
import pandas as pd
import numpy as np
import nltk
import re
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
stop = stopwords.words('russian')
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from collections import Counter
from imblearn.datasets import fetch_datasets
from sklearn.pipeline import make_pipeline
from imblearn.pipeline import make_pipeline as make_pipeline_imb
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss
from imblearn.metrics import classification_report_imbalanced
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier

In [3]:
def print_results(headline, true_value, pred):
    print(headline)
    print("accuracy: {}".format(accuracy_score(true_value, pred)))
    print("precision: {}".format(precision_score(true_value, pred)))
    print("recall: {}".format(recall_score(true_value, pred)))
    print("f1: {}".format(f1_score(true_value, pred)))

In [4]:
df =  pd.read_csv(r"C:\Users\coolz\assignments\data.csv", names = ['Review', 'Rating'], sep=',', dtype = {"Review": str, 'Rating': 'int32'})
df = df.dropna(axis=0)
np.random.seed(42)
df = df.drop_duplicates(subset=['Review', 'Rating'], keep=False)
df = df.reset_index(drop=True)

In [5]:
df['Rating']=df['Rating'].astype(int)
df=df[df['Rating']!=3]
df['label']=np.where(df['Rating']>=3,1,0)

In [6]:
df['Rating'].value_counts()

5    107885
4     50785
2     16121
1     15511
Name: Rating, dtype: int64

In [7]:
df = df.sample(frac=1).reset_index(drop=True) #shuffle
data = df[df['label']==0][:30000]
data = data.append(df[df['label']==1][:150000])
data = data.reset_index(drop=True)
display(data['label'].value_counts())
data

1    150000
0     30000
Name: label, dtype: int64

,Review,Rating,label
0,Насчет подружить с компьютером это целый анекд...,1,0
1,"Телефон чесно говно. Так серенький, простеньки...",2,0
2,Это был мой первый смартфон... сейчас купил Pr...,2,0
3,Полностью согласен с отзывом nnm Thursday (1 з...,2,0
4,Владею телефоном около 9 месяцев. Единственный...,2,0
...,...,...,...
179995,"Сравниваю данный апарат с прошлым Nova 2i, все...",4,1
179996,"Юзаю три или четыре года, сбился уже. Со своей...",5,1
179997,Отличный надежный телефон.,5,1
179998,Звук динамика ( как при разговоре так и при зв...,4,1


In [8]:
data['preprocess'] = data['Review'].apply(lambda x: " ".join(x.lower() for x in str(x).split()))
#data['preprocess'] = data['preprocess'].apply(lambda x: BeautifulSoup(x).get_text())
data['preprocess'] = data['preprocess'].apply(lambda x: re.sub(r"http\S+", "", x))
data["preprocess"] = data['preprocess'].str.replace('[^\w\s]','')
#data['preprocess'] = data['preprocess'].apply(lambda x: "".join([re.sub('[^A-Za-z]+','', x) for x in nltk.word_tokenize(x)]))
data['preprocess'] = data['preprocess'].apply(lambda x: " ".join([x for x in x.split() if x not in stop]))
data

C:\Users\coolz\AppData\Local\Temp/ipykernel_6080/3778976711.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  data["preprocess"] = data['preprocess'].str.replace('[^\w\s]','')


,Review,Rating,label,preprocess
0,Насчет подружить с компьютером это целый анекд...,1,0,насчет подружить компьютером это целый анекдот...
1,"Телефон чесно говно. Так серенький, простеньки...",2,0,телефон чесно говно серенький простенький советую
2,Это был мой первый смартфон... сейчас купил Pr...,2,0,это первый смартфон купил prestigio grace пока...
3,Полностью согласен с отзывом nnm Thursday (1 з...,2,0,полностью согласен отзывом nnm thursday 1 звез...
4,Владею телефоном около 9 месяцев. Единственный...,2,0,владею телефоном около 9 месяцев единственный ...
...,...,...,...,...
179995,"Сравниваю данный апарат с прошлым Nova 2i, все...",4,1,сравниваю данный апарат прошлым nova 2i параме...
179996,"Юзаю три или четыре года, сбился уже. Со своей...",5,1,юзаю четыре года сбился своей задачей прекрасн...
179997,Отличный надежный телефон.,5,1,отличный надежный телефон
179998,Звук динамика ( как при разговоре так и при зв...,4,1,звук динамика разговоре звонке вызывает раздра...


In [9]:
X_train,X_test,Y_train, Y_test = train_test_split(data['preprocess'], data['label'], test_size = 0.2, random_state = 142)
print("Train:",X_train.shape,Y_train.shape,"Test: ",(X_test.shape,Y_test.shape))

Train: (144000,) (144000,) Test:  ((36000,), (36000,))


In [10]:
vectorizer = TfidfVectorizer(dtype = np.float32)
vectorizer.fit(X_train.values.ravel())
tf_x_train = vectorizer.transform(X_train.values.ravel())
tf_x_test = vectorizer.transform(X_test.values.ravel())
#tf_x_train = tf_x_train.toarray()
#tf_x_test = tf_x_test.toarray()

In [11]:
data.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180000 entries, 0 to 179999
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Review      180000 non-null  object
 1   Rating      180000 non-null  int32 
 2   label       180000 non-null  int32 
 3   preprocess  180000 non-null  object
dtypes: int32(2), object(2)
memory usage: 485.1 MB


In [32]:
svm = LinearSVC(random_state=9742)
сlassifier = CalibratedClassifierCV(svm) 
#classifier = RandomForestClassifier

In [33]:
# build normal model with vectorizer
vectorized_normal_pipeline = make_pipeline(сlassifier)
vectorized_normal_model = vectorized_normal_pipeline.fit(tf_x_train, Y_train)
vectorized_normal_prediction = vectorized_normal_model.predict(tf_x_test)

In [34]:
# build model with SMOTE imblearn
smote_pipeline = make_pipeline_imb(SMOTE(random_state=42, k_neighbors = 5, n_jobs = 8), сlassifier)
smote_model = smote_pipeline.fit(tf_x_train, Y_train)
smote_prediction = smote_model.predict(tf_x_test)

In [35]:
# print information about both models
print("normal data distribution: {}".format(Counter(data['label'])))
X_smote, y_smote = SMOTE().fit_resample(tf_x_train, Y_train) #ругается на то что тут буквавки а не циферки, надо заранее векторизировать
print("SMOTE data distribution: {}".format(Counter(y_smote)))
#X_nearmiss, y_nearmiss = NearMiss().fit_sample(data['data'], data['target'])
#print("NearMiss data distribution: {}".format(Counter(y_nearmiss)))

normal data distribution: Counter({1: 150000, 0: 30000})
SMOTE data distribution: Counter({1: 119962, 0: 119962})


In [36]:
#from sklearn.linear_model import LogisticRegression

In [37]:
# build model with undersampling
#nr = NearMiss()
#X_train_miss, y_train_miss = nr.fit_resample(tf_x_train,Y_train.ravel())
#lr2 = LogisticRegression()
#lr2.fit(X_train_miss, y_train_miss.ravel())
#nearmiss_prediction = lr2.predict(tf_x_test)

In [38]:
# classification report
print(classification_report(Y_test, vectorized_normal_prediction))
print(classification_report_imbalanced(Y_test, smote_prediction))

              precision    recall  f1-score   support

           0       0.85      0.75      0.80      5962
           1       0.95      0.97      0.96     30038

    accuracy                           0.94     36000
   macro avg       0.90      0.86      0.88     36000
weighted avg       0.93      0.94      0.94     36000

                   pre       rec       spe        f1       geo       iba       sup

          0       0.75      0.77      0.95      0.76      0.85      0.72      5962
          1       0.95      0.95      0.77      0.95      0.85      0.74     30038

avg / total       0.92      0.92      0.80      0.92      0.85      0.74     36000



In [39]:
print('vectorized_normal Pipline Score {}'.format(vectorized_normal_pipeline.score(tf_x_test, Y_test)))
print('SMOTE Pipeline Score {}'.format(smote_pipeline.score(tf_x_test, Y_test)))
#print('NearMiss Pipeline Score {}'.format(nearmiss_pipeline.score(tf_x_test, Y_test)))

vectorized_normal Pipline Score 0.9192222222222223
SMOTE Pipeline Score 0.9192222222222223


In [40]:
print_results("vectorized normal classification", Y_test, vectorized_normal_prediction)
print_results("SMOTE classification", Y_test, smote_prediction)
#print_results("NearMiss classification", Y_test, nearmiss_prediction)

vectorized normal classification
accuracy: 0.9369166666666666
precision: 0.9519221379512386
recall: 0.9735668153672016
f1: 0.9626228213104231
SMOTE classification
accuracy: 0.9192222222222223
precision: 0.9538914541925986
recall: 0.9490645182768493
f1: 0.9514718643615246


In [41]:
roc_auc_score(Y_test, smote_prediction)

0.8589670125768681

In [42]:
roc_auc_score(Y_test, vectorized_normal_prediction)

0.862915578096214

In [43]:
print(tf_x_test)

  (0, 323729)	0.23281614
  (0, 314514)	0.14253204
  (0, 288978)	0.15453464
  (0, 266879)	0.22037578
  (0, 245873)	0.32021886
  (0, 230846)	0.22896151
  (0, 215483)	0.32842496
  (0, 209854)	0.2124623
  (0, 209564)	0.17283924
  (0, 194585)	0.13380037
  (0, 189380)	0.16487402
  (0, 179847)	0.23800775
  (0, 179553)	0.12949972
  (0, 151303)	0.12678376
  (0, 130455)	0.15718016
  (0, 119977)	0.15202004
  (0, 94134)	0.25511754
  (0, 59813)	0.20476684
  (0, 57565)	0.2589254
  (0, 50494)	0.14977527
  (0, 45479)	0.1061281
  (0, 28855)	0.24675159
  (0, 9143)	0.2533025
  (1, 311000)	0.12975878
  (1, 298494)	0.41827714
  :	:
  (35999, 239031)	0.15937507
  (35999, 237820)	0.2164202
  (35999, 234435)	0.17802644
  (35999, 233566)	0.16654307
  (35999, 230508)	0.11018709
  (35999, 209718)	0.10960243
  (35999, 206534)	0.12476412
  (35999, 196448)	0.19131576
  (35999, 188873)	0.13912702
  (35999, 178796)	0.2134171
  (35999, 143674)	0.15268235
  (35999, 143301)	0.219926
  (35999, 138482)	0.25570175
  (35999

In [28]:
custom_text = ["самый лучший телефон на свете!!! всем определённо советую"]
custom_text = vectorizer.transform(custom_text)

In [44]:
vectorized_normal_model.predict_proba(custom_text)

array([[1.03016848e-04, 9.99896983e-01]])